In [52]:
# Team YOLO
# 19.11.2022
# Amol Kodange
# Mohit Shah
# Srishti Mishra

In [46]:
import pandas as pd
import numpy as np
import shutil
from pathlib import PurePath
import tensorflow as tf
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
#from sklearn import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import splitfolders

In [47]:
df = pd.read_csv("train.csv")
df.head()

,label,latitude,longitude,year,example_path
0,0,-2.051853,111.826093,2001,train_test_data/train/1297.png
1,2,-1.989349,105.309496,2013,train_test_data/train/1199.png
2,0,1.223256,100.702217,2014,train_test_data/train/1348.png
3,0,-2.342948,103.890226,2008,train_test_data/train/2214.png
4,0,-0.126555,101.758175,2011,train_test_data/train/2220.png


In [17]:
#df_np = np.asarray(df)
#labels = df_np[:,0]
#paths = df_np[:,4]
#print(paths[0])

#for element,x in enumerate(labels):
#    p = PurePath(paths[element])
#    spam = list(p.parts)
#    spam.pop()
#    if x == 0:
#        spam.append('Class 0')
#    elif x == 1:
#        spam.append('Class 1')
#    else:
#        spam.append('Class 2')
#    new_path = PurePath('').joinpath(*spam)
#    shutil.move(paths[element], new_path)

In [34]:
splitfolders.ratio(r"/content/drive/MyDrive/Hackathon/train_test_data/train",output="/content/drive/MyDrive/Hackathon/train_test_data/train",seed=42,ratio=(0.8,0.2,0.0), group_prefix=None,move=False)

Copying files: 1714 files [00:17, 100.01 files/s]


In [48]:
DATASET_PATH  = r"/content/drive/MyDrive/Hackathon/train_test_data"
IMAGE_SIZE  = (332, 332)
NUM_CLASSES = 3
BATCH_SIZE   = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 20
WEIGHTS_FINAL = 'model-inception_resnet_v2-final.h5'

In [49]:
net = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

x = net.output

x = Flatten()(x)
x = Dropout(0.5)(x)

output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

net_final = Model(inputs=net.input, outputs=output_layer)

for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
#print(net_final.summary())

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_df = DATASET_PATH + '/train/train'
val_df = DATASET_PATH + '/train/val'
train_batches = train_datagen.flow_from_directory(train_df,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(val_df,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Found 1370 images belonging to 3 classes.
Found 342 images belonging to 3 classes.


In [50]:
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/20
171/171 [==============================] - 93s 441ms/step - loss: 1.0680 - accuracy: 0.5264 - val_loss: 0.9770 - val_accuracy: 0.5536
Epoch 2/20
171/171 [==============================] - 73s 428ms/step - loss: 0.9323 - accuracy: 0.6028 - val_loss: 0.8536 - val_accuracy: 0.6071
Epoch 3/20
171/171 [==============================] - 73s 423ms/step - loss: 0.8848 - accuracy: 0.6322 - val_loss: 0.7607 - val_accuracy: 0.6815
Epoch 4/20
171/171 [==============================] - 72s 422ms/step - loss: 0.8033 - accuracy: 0.6755 - val_loss: 0.7106 - val_accuracy: 0.7232
Epoch 5/20
171/171 [==============================] - 73s 427ms/step - loss: 0.7671 - accuracy: 0.6916 - val_loss: 0.7283 - val_accuracy: 0.7292
Epoch 6/20
171/171 [==============================] - 72s 421ms/step - loss: 0.7537 - accuracy: 0.6909 - val_loss: 0.6992 - val_accuracy: 0.7083
Epoch 7/20
171/171 [==============================] - 73s 427ms/step - loss: 0.7290 - accuracy: 0.7048 - val_loss: 0.6431 - val_ac

In [51]:
Y_pred = net_final.predict_generator(valid_batches,valid_batches.samples// BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(valid_batches.classes, y_pred))
print('Classification Report')
target_names = ['0','1','2']
print(classification_report(valid_batches.classes, y_pred, target_names=target_names))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


Confusion Matrix
[[95 25 52]
 [19  4 16]
 [77 12 42]]
Classification Report
              precision    recall  f1-score   support

           0       0.50      0.55      0.52       172
           1       0.10      0.10      0.10        39
           2       0.38      0.32      0.35       131

    accuracy                           0.41       342
   macro avg       0.33      0.33      0.32       342
weighted avg       0.41      0.41      0.41       342



# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
